## Find the shared Account network among the following the stream seeded accounts
* how much overlap is there in the people followed by seed accounts? 

## Tweet Processing
This uses utility code stored in `tweetscape-experiments/python/tweet_processing`
This utility library will mold as we do more experiments.

#### install 
install commands (assumgin you start from teh current directory)
```
cd ../../tweet_processing
pip install -e .
```

#### Loading data 

Instantiate a StreamTweetProcessor object with the twarc client and your data_dir. 

The code for this is in the next cell

@andre: Make sure you point the data_dir varialbe to whatever path you have the unzipped file I sent you
The data dir is strucutred like this:
```
data/
    group_name/
        tweets.csv
        ref_tweets.csv
        following.csv
    group_name2/
        tweets.csv
        ref_tweets.csv
        following.csv
```

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from twarc import Twarc2, expansions
from tweet_processing import StreamTweetProcessor

load_dotenv()

twarc_client = Twarc2(
    consumer_key=os.environ["consumer_key"], 
    consumer_secret=os.environ["consumer_secret"],
    access_token=os.environ["access_token"], 
    access_token_secret=os.environ["access_token_secret"]
)

data_dir="../stream-seeding/data" 

tp = StreamTweetProcessor(twarc_client=twarc_client, data_dir=data_dir)


# group_name = "longevity-pranab"
group_name = "CA-Abundance-Economy"

df_following, df_tweets, df_ref_tweets = tp.load_stream_seed_data(group_name) 


/home/nick/Documents/GitHub/tweetscape-experiments/python/tweet_processing/tweet_processing/stream_tweet_processor.py:55: DtypeWarning: Columns (45,46,55,62,63,64,65,66,67,68,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.read_csv(f"{dir_}/tweets.csv")


In [2]:
## Get a set of usernames that each seed account follows
users_following = {}
for user in df_following["referencer.username"].unique().tolist():
    users_following[user] = set(df_following[df_following["referencer.username"] == user]["username"].tolist())
    
## index will be useful to access metadata (followers_count) about these accounts
df_following.set_index("username", inplace=True)


### Create a dataframe that will show the following_overlap for each of the accounts followed by at least 1 of the seed accounts. 
### We are most interested in the intersection among the following of those accounts
### If the accounts are picked well, they will likely have interesting intersection among their shared interest

following_overlap = {}

for stream_user, following in users_following.items():
    for followed_user in following: 
        following_overlap.setdefault(followed_user, []).append(stream_user)

df_data = []

for followed_user, stream_users in following_overlap.items():
    num_followers_of_followed = df_following.loc[followed_user]["public_metrics.followers_count"]
    if isinstance(num_followers_of_followed, pd.Series):
        num_followers_of_followed = num_followers_of_followed.iloc[0]
    df_data.append([followed_user, stream_users, len(stream_users), num_followers_of_followed, f"https://twitter.com/{followed_user}"])
    
overlap_df = pd.DataFrame(df_data, columns=["followed.username", "stream_users", "num_stream_users", "num_followers_of_followed", "profile_link"])
overlap_df["num_stream_users"].value_counts()

1    7097
2     393
3      33
Name: num_stream_users, dtype: int64

### Show Overlap
Here, we show users that are followed by multiple seed accounts, sorted by the number of followers of that accounts. 

Filtering out to the lower follower accounts are a signal that that person talks abou their shared interest, and is an easy way to avoid constant reccomendation of accounts like Elon

In [3]:
NUM_SEED_ACCOUNTS = 3
overlap_df[overlap_df["num_stream_users"]==NUM_SEED_ACCOUNTS].sort_values("num_followers_of_followed")

,followed.username,stream_users,num_stream_users,num_followers_of_followed,profile_link
2304,gillibits,"[rklau, anniefryman, hanlonbt]",3,1380,https://twitter.com/gillibits
3711,thuddwhirr,"[rklau, anniefryman, hanlonbt]",3,1790,https://twitter.com/thuddwhirr
870,mishachellam,"[rklau, anniefryman, hanlonbt]",3,4328,https://twitter.com/mishachellam
1399,tchu88,"[rklau, anniefryman, hanlonbt]",3,8278,https://twitter.com/tchu88
2526,cbracy,"[rklau, anniefryman, hanlonbt]",3,8297,https://twitter.com/cbracy
3663,allafarce,"[rklau, anniefryman, hanlonbt]",3,8788,https://twitter.com/allafarce
3701,profjohnapowell,"[rklau, anniefryman, hanlonbt]",3,9165,https://twitter.com/profjohnapowell
4470,Lenny_Mendonca,"[rklau, anniefryman, hanlonbt]",3,10171,https://twitter.com/Lenny_Mendonca
515,NancySkinnerCA,"[rklau, anniefryman, hanlonbt]",3,12838,https://twitter.com/NancySkinnerCA
1888,starsandrobots,"[rklau, anniefryman, hanlonbt]",3,21575,https://twitter.com/starsandrobots
